## make sure you have all libraries installed. 
use conda environment: /scratch/da2734/twitter/worldbank_twitter_environment.yml

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# loading random samples

In [2]:
# random contains 0.8G of data!!
import time
start_time = time.time()
import pyarrow.parquet as pq
from glob import glob
import os
country_code = 'US'
month = '2012-1'
path_to_data = '/scratch/spf248/twitter/data/classification/US/random/'
tweets_random=pq.ParquetDataset(glob(os.path.join(path_to_data,                                           
#                                            country_code,
#                                            month,
                                           '*.parquet'))).read().to_pandas()
tweets_random['tweet_id'] = tweets_random['tweet_id'].astype(int)
print('time taken to load random sample:', str(time.time() - start_time), 'seconds')
print(tweets_random.shape)

time taken to load random sample: 440.7797255516052 seconds
(92114009, 11)


In [3]:
start_time = time.time()
tweets_random = tweets_random[['tweet_id', 'text']]
print('drop columns:', str(time.time() - start_time), 'seconds')

drop columns: 5.32327938079834 seconds


In [4]:
tweets_random.head()

,tweet_id,text
0,367881326273105920,@shoebydoo32 I only left to go back home for t...
1,367881326281519105,oh my god bina id idnt read that u played GTA ...
2,367882185916702722,I Have To Make What I Think Is The Best Decisi...
3,367883553121394690,@elizrod_ that's from hard work 😉
4,367883867719348224,RT @MileenaSucks: Can I just lay out in the gr...


# random top 5M files 

In [5]:
# outputting 1M (pinky finger up) tweets by column
import glob
import pandas as pd
import time
import pickle
import matplotlib.pyplot as plt
from scipy.special import softmax
import numpy as np


# boundary_list = [i/20.0 for i in range(11)]
# boundary_list.reverse()
# print('boundary_list', boundary_list)

for column in ["is_unemployed", "lost_job_1mo", "job_search", "is_hired_1mo", "job_offer"]:
    print('\n\n', column)
    
#     if column != 'job_offer': continue
    
#     start_time = time.time()
    # model_output_path = '/scratch/da2734/twitter/jobs/running_on_200Msamples/pred_output_100M_ONNX_optimized/{}/'.format(column)
#     print(model_output_path)

    start_time = time.time()
    model_output_random1 = pd.concat([pd.read_csv(f) for f in glob.glob(
        '/scratch/da2734/twitter/jobs/running_on_200Msamples/iteration1/glove_inferences/glove_inferences_dhaval/{}/'.format(column)+
        'random*.csv')], ignore_index = True)
    print('dhaval time taken to load random sample:', str(time.time() - start_time), 'seconds', model_output_random1.shape)
    
    start_time = time.time()
    model_output_random2 = pd.concat([pd.read_csv(f) for f in glob.glob(
        '/scratch/da2734/twitter/jobs/running_on_200Msamples/iteration1/glove_inferences/glove_inferences_sam/{}/'.format(column)+
        'random*.csv')], ignore_index = True)
    print('sam time taken to load random sample:', str(time.time() - start_time), 'seconds', model_output_random2.shape)

    start_time = time.time()
    model_output_random = pd.concat([model_output_random1, model_output_random2])
    print('concat:', str(time.time() - start_time), 'seconds')

    start_time = time.time()
    model_output_random = model_output_random[['tweet_id', 'proba']]
    print('drop columns:', str(time.time() - start_time), 'seconds')    

    start_time = time.time()
    merged_random = pd.merge(model_output_random, tweets_random, how='inner', on = 'tweet_id')
    print('time taken to merge:', str(time.time() - start_time), 'seconds', merged_random.shape)

    # print(merged_random.head())

    start_time = time.time()
    merged_random = merged_random.drop_duplicates(['text'])
    print('drop dup:', str(time.time() - start_time), 'seconds', merged_random.shape)
    
    start_time = time.time()
    merged_random = merged_random.sort_values(by=['proba'], ascending=False)
    print('sort:', str(time.time() - start_time), 'seconds', merged_random.shape)    
    
    start_time = time.time()
    merged_random_temp = merged_random[:5000000]
    print('top 10M truncate:', str(time.time() - start_time), 'seconds', merged_random.shape)   
    
    print(merged_random_temp.tail(50))
    
    start_time = time.time()
    pickle.dump( merged_random_temp, open( "./ONNXturk100M/iteration1/{}_GLOVECNN_merged_random_100m_july1.pkl".format(column), "wb" ) )
    print('time taken to dump:', str(time.time() - start_time), 'seconds')
        
#     break




 is_unemployed
dhaval time taken to load random sample: 72.79461789131165 seconds (45403146, 2)
sam time taken to load random sample: 70.71351265907288 seconds (46139942, 2)
concat: 1.076456069946289 seconds
drop columns: 0.6384522914886475 seconds
time taken to merge: 92.41594290733337 seconds (91543088, 3)
drop dup: 132.29338812828064 seconds (84063455, 3)
sort: 58.255332708358765 seconds (84063455, 3)
top 10M truncate: 0.0006420612335205078 seconds (84063455, 3)
                    tweet_id     proba  \
54503667  726772870114861057  0.315428   
67056185  636011684159553536  0.315428   
36193661  509522867193532416  0.315428   
90195797  681217225940848640  0.315428   
5859759   614623305014857728  0.315428   
1537069   618943039407439873  0.315428   
26144902  657039147975086080  0.315428   
4209031   908119920118059008  0.315428   
21620326  541697172123578368  0.315428   
79419446  279764170675720192  0.315428   
11549360  691379139320897537  0.315428   
84431785  55073422771385

time taken to dump: 3.6084368228912354 seconds


 job_search
dhaval time taken to load random sample: 79.00391340255737 seconds (45403146, 2)
sam time taken to load random sample: 76.98716473579407 seconds (46139942, 2)
concat: 1.1033446788787842 seconds
drop columns: 0.6489970684051514 seconds
time taken to merge: 93.70298218727112 seconds (91543088, 3)
drop dup: 124.26292729377747 seconds (84063455, 3)
sort: 63.569708824157715 seconds (84063455, 3)
top 10M truncate: 4.127707242965698 seconds (84063455, 3)
                     tweet_id     proba  \
34197908   900651169118400512  0.206155   
40756034   603191989656526849  0.206155   
78032596   728501531633319937  0.206155   
79711005   725808905717043200  0.206155   
25895934   743196049246826497  0.206155   
14888869   656264698816393216  0.206155   
46285190   518963092168773632  0.206155   
34105429   647181086330503169  0.206155   
12277609   639277361079435264  0.206155   
37040356   779216501249236992  0.206155   
70057006  1004

time taken to dump: 3.8029568195343018 seconds


 job_offer
dhaval time taken to load random sample: 79.67919206619263 seconds (45403146, 2)
sam time taken to load random sample: 82.5118670463562 seconds (46139942, 2)
concat: 1.2263267040252686 seconds
drop columns: 0.665715217590332 seconds
time taken to merge: 94.20846009254456 seconds (91543088, 3)
drop dup: 125.87841057777405 seconds (84063455, 3)
sort: 62.68632531166077 seconds (84063455, 3)
top 10M truncate: 3.8062336444854736 seconds (84063455, 3)
                     tweet_id     proba  \
69711976   560282535587237888  0.369386   
61720049   644322030662234112  0.369386   
3735268    533044457164537857  0.369386   
25730182   574340504499851264  0.369386   
33616259   283563681474818048  0.369386   
54097201   594118166508400640  0.369386   
25740199   588703890285187072  0.369386   
87331336   724981461942677504  0.369386   
66644943   766760454307753984  0.369386   
39713218   765915352295276544  0.369386   
36234507   442542

# filtered top 10M files

In [ ]:
print(merged_filtered.shape)

In [ ]:
# outputting 1M (pinky finger up) tweets by column
import glob
import pandas as pd
import time
import pickle
import matplotlib.pyplot as plt
from scipy.special import softmax
import numpy as np


# boundary_list = [i/20.0 for i in range(11)]
# boundary_list.reverse()
# print('boundary_list', boundary_list)

for column in ["is_unemployed", "lost_job_1mo", "job_search", "is_hired_1mo", "job_offer"]:
    print('\n\n', column)
    
#     if column == 'job_offer': continue
    
    start_time = time.time()
    model_output_path = '/scratch/da2734/twitter/jobs/running_on_200Msamples/pred_output_100M_GLOVE_CNN_optimized/{}/'.format(column)
    print(model_output_path)

    start_time = time.time()
    model_output_filtered = pd.concat([pd.read_csv(f) for f in glob.glob(model_output_path+'filtered*.csv')], ignore_index = True)
    print('time taken to load filtered sample:', str(time.time() - start_time), 'seconds', model_output_filtered.shape)
    
    start_time = time.time()
    model_output_filtered = model_output_filtered[['tweet_id', 'proba']]
    print('drop columns:', str(time.time() - start_time), 'seconds')
    

    start_time = time.time()
    merged_filtered = pd.merge(model_output_filtered, tweets_filtered, how='inner', on = 'tweet_id')
    print('time taken to merge:', str(time.time() - start_time), 'seconds', merged_filtered.shape)

    print(merged_filtered.shape)

    start_time = time.time()
    merged_filtered = merged_filtered.drop_duplicates(['text'])
    print('drop dup:', str(time.time() - start_time), 'seconds', merged_filtered.shape)
    
    start_time = time.time()
    merged_filtered = merged_filtered.sort_values(by=['proba'], ascending=False)
    print('sort:', str(time.time() - start_time), 'seconds', merged_filtered.shape)    
    
    start_time = time.time()
    merged_filtered_temp = merged_filtered[:5000000]
    print('top 5M truncate:', str(time.time() - start_time), 'seconds', merged_filtered.shape)   
    
    print(merged_filtered_temp.tail(50)['text'])
    
    start_time = time.time()
    pickle.dump( merged_filtered_temp, open( "./ONNXturk100M/{}_GLOVE_CNN_merged_filtered_100m_jun22.pkl".format(column), "wb" ) )
    print('time taken to dump:', str(time.time() - start_time), 'seconds')
        
#     break


In [ ]:
start_time = time.time()
merged_filtered_temp = merged_filtered[5000000:12000000]
print('top 5M truncate:', str(time.time() - start_time), 'seconds', merged_filtered.shape)   

# print(merged_filtered_temp.tail(50)['text'])

start_time = time.time()
pickle.dump( merged_filtered_temp, open( "./ONNXturk100M/{}_GLOVE_CNN_merged_filtered_100m_jun22_5_to_12M.pkl".format(column), "wb" ) )
print('time taken to dump:', str(time.time() - start_time), 'seconds')



# random top 10M files

In [ ]:
# outputting 1M (pinky finger up) tweets by column
import glob
import pandas as pd
import time
import pickle
import matplotlib.pyplot as plt
from scipy.special import softmax
import numpy as np


# boundary_list = [i/20.0 for i in range(11)]
# boundary_list.reverse()
# print('boundary_list', boundary_list)

for column in ["is_unemployed", "lost_job_1mo", "job_search", "is_hired_1mo", "job_offer"]:
    print('\n\n', column)
    
    if column != 'job_offer': continue
    
    start_time = time.time()
    model_output_path = '/scratch/da2734/twitter/jobs/running_on_200Msamples/pred_output_100M_GLOVE_CNN_optimized/{}/'.format(column)
    print(model_output_path)

    start_time = time.time()
    model_output_random = pd.concat([pd.read_csv(f) for f in glob.glob(model_output_path+'random*.csv')], ignore_index = True)
    print('time taken to load random sample:', str(time.time() - start_time), 'seconds', model_output_random.shape)
    
    start_time = time.time()
    model_output_random = model_output_random[['tweet_id', 'proba']]
    print('drop columns:', str(time.time() - start_time), 'seconds')
    

    start_time = time.time()
    merged_random = pd.merge(model_output_random, tweets_random, how='inner', on = 'tweet_id')
    print('time taken to merge:', str(time.time() - start_time), 'seconds', merged_random.shape)

#     print(merged_random.head())

    start_time = time.time()
    merged_random = merged_random.drop_duplicates(['text'])
    print('drop dup:', str(time.time() - start_time), 'seconds', merged_random.shape)
    
    start_time = time.time()
    merged_random = merged_random.sort_values(by=['proba'], ascending=False)
    print('sort:', str(time.time() - start_time), 'seconds', merged_random.shape)    
    
#     start_time = time.time()
#     merged_random_temp = merged_random[:5000000]
#     print('top 5M truncate:', str(time.time() - start_time), 'seconds', merged_random.shape)   
    
#     print(merged_random_temp.tail(50)['text'])
    
#     start_time = time.time()
#     pickle.dump( merged_random_temp, open( "./ONNXturk100M/{}_GLOVE_CNN_merged_random_100m_jun22.pkl".format(column), "wb" ) )
#     print('time taken to dump:', str(time.time() - start_time), 'seconds')
        
#     break


In [ ]:
# outputting 1M (pinky finger up) tweets by column
import glob
import pandas as pd
import time
import pickle
import matplotlib.pyplot as plt
from scipy.special import softmax
import numpy as np


for column in ["is_unemployed", "lost_job_1mo", "job_search", "is_hired_1mo", "job_offer"]:
    print('\n\n', column)
    
    start_time = time.time()
    model_output_path = '/scratch/da2734/twitter/jobs/running_on_200Msamples/pred_output_100M_GLOVE_CNN_optimized/{}/'.format(column)
    print(model_output_path)

    start_time = time.time()
    model_output_filtered = pd.concat([pd.read_csv(f) for f in glob.glob(model_output_path+'filtered*.csv')], ignore_index = True)
    print('time taken to load filtered sample:', str(time.time() - start_time), 'seconds', model_output_filtered.shape)
    
    start_time = time.time()
    model_output_filtered = model_output_filtered[['tweet_id', 'proba']]
    model_output_filtered.columns = ['tweet_id', column]
    print('drop columns:', str(time.time() - start_time), 'seconds')
    

    if column == 'is_unemployed':
        start_time = time.time()
        merged_filtered = pd.merge(model_output_filtered, tweets_filtered, how='inner', on = 'tweet_id')
        print('time taken to merge:', str(time.time() - start_time), 'seconds', merged_filtered.shape)
    else:
        start_time = time.time()
        merged_filtered = pd.merge(model_output_filtered, merged_filtered, how='inner', on = 'tweet_id')
        print('time taken to merge:', str(time.time() - start_time), 'seconds', merged_filtered.shape)
            
    print(merged_filtered.tail(10))
            
#     break
    
    
start_time = time.time()
merged_filtered = merged_filtered.drop_duplicates(['text'])
print('drop dup:', str(time.time() - start_time), 'seconds', merged_filtered.shape)

start_time = time.time()
pickle.dump( merged_filtered, open( "./ONNXturk100M/ALL_ONNX_BERT_ST_merged_filtered_100m_jun24.pkl", "wb" ) )
print('time taken to dump:', str(time.time() - start_time), 'seconds')
        


# merging random tweets into one big file

In [ ]:
# outputting 1M (pinky finger up) tweets by column
import glob
import pandas as pd
import time
import pickle
import matplotlib.pyplot as plt
from scipy.special import softmax
import numpy as np


for column in ["is_unemployed", "lost_job_1mo", "job_search", "is_hired_1mo", "job_offer"]:
    print('\n\n', column)
    
    start_time = time.time()
    model_output_path = '/scratch/da2734/twitter/jobs/running_on_200Msamples/pred_output_100M_GLOVE_CNN_optimized/{}/'.format(column)
    print(model_output_path)

    start_time = time.time()
    model_output_random = pd.concat([pd.read_csv(f) for f in glob.glob(model_output_path+'random*.csv')], ignore_index = True)
    print('time taken to load random sample:', str(time.time() - start_time), 'seconds', model_output_random.shape)
    
    start_time = time.time()
    model_output_random = model_output_random[['tweet_id', 'proba']]
    model_output_random.columns = ['tweet_id', column]
    print('drop columns:', str(time.time() - start_time), 'seconds')
    

    if column == 'is_unemployed':
        start_time = time.time()
        merged_random = pd.merge(model_output_random, tweets_random, how='inner', on = 'tweet_id')
        print('time taken to merge:', str(time.time() - start_time), 'seconds', merged_random.shape)
    else:
        start_time = time.time()
        merged_random = pd.merge(model_output_random, merged_random, how='inner', on = 'tweet_id')
        print('time taken to merge:', str(time.time() - start_time), 'seconds', merged_random.shape)
            
    print(merged_random.tail(10))
            
#     break
    
    
start_time = time.time()
merged_random = merged_random.drop_duplicates(['text'])
print('drop dup:', str(time.time() - start_time), 'seconds', merged_random.shape)

start_time = time.time()
pickle.dump( merged_random, open( "./ONNXturk100M/ALL_GLOVE_CNN_merged_random_100m_jun24.pkl", "wb" ) )
print('time taken to dump:', str(time.time() - start_time), 'seconds')
        


In [ ]:
model_output_random.head()